In [1]:
import tensorflow as tf
import numpy as np

In [45]:
def dictionaryGen(D):
    
    """ Generate and return 
        - D integers on the x axis evenly distributed around 0 (??? w/ abs value \leq 100 ???)
        - The step size Δx """

    d_pos = np.linspace(0, 100, num= int(D/2), endpoint=False, retstep=True, dtype=np.int32)
    d_neg = (-1*d_pos[0][1:]) # get the negative dictionary points  
    return (d_pos[1], np.concatenate((np.sort(d_neg), d_pos[0]), axis=None))

In [49]:
step, d = dictionaryGen(50)
tf.constant(d)

<tf.Tensor: shape=(49,), dtype=int32, numpy=
array([-96, -92, -88, -84, -80, -76, -72, -68, -64, -60, -56, -52, -48,
       -44, -40, -36, -32, -28, -24, -20, -16, -12,  -8,  -4,   0,   4,
         8,  12,  16,  20,  24,  28,  32,  36,  40,  44,  48,  52,  56,
        60,  64,  68,  72,  76,  80,  84,  88,  92,  96], dtype=int32)>

In [83]:
from tensorflow.keras.layers import Layer

class Kaf(Layer):

    def __init__(self, D):

        super(Kaf, self).__init__()
        
        # Initialize constants
        self.D = D
        step, dict = dictionaryGen(D)
        self.d = tf.constant(dict)
        self.k_bandw = 1/(6*(np.power(step,2)))
        

    def build(self, input_shape):
        
        # Mixing coefficients sampled from a normal dist
        self.a = self.add_weight(shape=(input_shape[-1], self.D), 
                                 initializer= 'random_normal',
                                 trainable=True)

    def call(self, inputs):
        print(self.a)

         


In [84]:
x = tf.random.uniform(shape = (20, ), maxval=1000, dtype=tf.float32)

kaf = Kaf(20)

kaf(x)

<tf.Variable 'kaf_17/Variable:0' shape=(20, 20) dtype=float32, numpy=
array([[ 3.82533148e-02,  3.61576863e-02,  5.30118160e-02,
        -4.39522676e-02,  2.83939894e-02,  1.22113852e-02,
         4.51750681e-02, -2.72306986e-02, -1.15551986e-01,
         2.80824397e-02, -1.84931144e-01, -1.48511454e-01,
         8.84378105e-02, -1.29018119e-02, -3.17439251e-02,
         2.39584073e-02, -3.83792333e-02,  4.62739207e-02,
         2.79773083e-02,  7.93503318e-03],
       [ 5.17101660e-02,  5.78577518e-02,  3.01747117e-02,
         6.00397699e-02,  2.80674896e-03,  1.91389862e-03,
         4.64330614e-02, -2.81738606e-03, -3.28963064e-02,
        -3.83690558e-02, -4.43826206e-02,  4.08481322e-02,
        -2.99476087e-02,  1.75428148e-02, -3.84399891e-02,
        -7.51875713e-03,  1.03275059e-02, -4.53918092e-02,
         7.97440410e-02, -5.67155741e-02],
       [ 1.76322442e-02, -1.97292236e-03, -1.27641307e-02,
        -5.58144413e-02,  1.42216161e-01, -5.85675649e-02,
        -6.0243021